In [1]:
import numpy as np
import scipy
import pandas as pd
import os
import warnings
warnings.filterwarnings('ignore')
from sklearn.preprocessing import MinMaxScaler, StandardScaler, OneHotEncoder
from sklearn.base import clone

# imports for visualizations
import matplotlib.pyplot as plt
import seaborn as sns
from math import ceil
from itertools import product
from pandas_profiling import ProfileReport
%matplotlib inline
%config InlineBackend.figure_format = 'retina' 
import matplotlib.cm as cm
sns.set()

from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import MinMaxScaler, StandardScaler, OneHotEncoder
from sklearn.neighbors import LocalOutlierFactor
from sklearn.impute import KNNImputer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.metrics import confusion_matrix, roc_auc_score, roc_curve, classification_report, precision_recall_curve
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.model_selection import RepeatedStratifiedKFold

from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LinearRegression
from imblearn.over_sampling import SMOTENC
from collections import Counter
from scikitplot.metrics import plot_cumulative_gain, plot_lift_curve
from sklearn.model_selection import RandomizedSearchCV 

In [2]:
df = pd.read_csv("H2.csv")

In [4]:
df.head(5)

,IsCanceled,LeadTime,ArrivalDateYear,ArrivalDateMonth,ArrivalDateWeekNumber,ArrivalDateDayOfMonth,StaysInWeekendNights,StaysInWeekNights,Adults,Children,...,DepositType,Agent,Company,DaysInWaitingList,CustomerType,ADR,RequiredCarParkingSpaces,TotalOfSpecialRequests,ReservationStatus,ReservationStatusDate
0,0,6,2015,July,27,1,0,2,1,0.0,...,No Deposit,6,NULL,0,Transient,0.0,0,0,Check-Out,2015-07-03
1,1,88,2015,July,27,1,0,4,2,0.0,...,No Deposit,9,NULL,0,Transient,76.5,0,1,Canceled,2015-07-01
2,1,65,2015,July,27,1,0,4,1,0.0,...,No Deposit,9,NULL,0,Transient,68.0,0,1,Canceled,2015-04-30
3,1,92,2015,July,27,1,2,4,2,0.0,...,No Deposit,9,NULL,0,Transient,76.5,0,2,Canceled,2015-06-23
4,1,100,2015,July,27,2,0,2,2,0.0,...,No Deposit,9,NULL,0,Transient,76.5,0,1,Canceled,2015-04-02


In [7]:
df = df.copy()
drop_features = ['ArrivalDateYear','ADR','ArrivalDateMonth','Agent','Company','AssignedRoomType','ReservationStatusDate','ReservationStatus']
keep_features = df.columns.drop(drop_features).to_list()
df_1 = df[keep_features]

In [8]:
categorical_1 = ['Country','CustomerType','DepositType','DistributionChannel','MarketSegment','Meal',\
               'IsRepeatedGuest','ReservedRoomType']
integer_variables_1= ["LeadTime","ArrivalDateDayOfMonth", "StaysInWeekendNights",\
                     "StaysInWeekNights", "Adults", "Children", "Babies","DaysInWaitingList",\
                    'BookingChanges','PreviousBookingsNotCanceled','PreviousCancellations',\
                    'RequiredCarParkingSpaces','TotalOfSpecialRequests']

In [9]:
# Use OneHotEncoder to encode the categorical features. Get feature names and create a DataFrame 
# with the one-hot encoded categorical features (pass feature names)
ohc = OneHotEncoder(sparse=False, drop="first")
ohc_feat = ohc.fit_transform(df_1[categorical_1])
ohc_feat_names = ohc.get_feature_names()
ohc_df = pd.DataFrame(ohc_feat, index=df_1.index, columns=ohc_feat_names)
ohc_df.head()

,x0_AGO,x0_AIA,x0_ALB,x0_AND,x0_ARE,x0_ARG,x0_ARM,x0_ASM,x0_ATA,x0_ATF,...,x5_HB,x5_SC,x6_1,x7_B,x7_C,x7_D,x7_E,x7_F,x7_G,x7_P
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
# Reassigning df to contain ohc variables
df_1_ohc = pd.concat([df_1.drop(columns=categorical_1), ohc_df], axis=1)
df_1_ohc.head()

,IsCanceled,LeadTime,ArrivalDateWeekNumber,ArrivalDateDayOfMonth,StaysInWeekendNights,StaysInWeekNights,Adults,Children,Babies,PreviousCancellations,...,x5_HB,x5_SC,x6_1,x7_B,x7_C,x7_D,x7_E,x7_F,x7_G,x7_P
0,0,6,27,1,0,2,1,0.0,0,0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,88,27,1,0,4,2,0.0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1,65,27,1,0,4,1,0.0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1,92,27,1,2,4,2,0.0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1,100,27,2,0,2,2,0.0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
target = ["IsCanceled"]
features= df_1_ohc.columns.drop(target).to_list()
data =df_1_ohc[features]
target= df_1_ohc[target]

In [12]:
lof = LocalOutlierFactor(contamination = 0.05)
lof_out = pd.Series(lof.fit_predict(df_1_ohc[integer_variables_1]), index = df_1_ohc[integer_variables_1].index)
lof_out.value_counts(normalize = True)

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').